In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import copy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [6]:
client_id = open('client.txt').readlines()[0]
secret = open('secret.txt').readlines()[0]
username = open('username.txt').readlines()[0]
redirect_url = open('redirect.txt').readlines()[0]

In [7]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=secret) 
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
scope = 'user-library-read playlist-read-private user-top-read playlist-modify-public'
token = util.prompt_for_user_token(username, scope,client_id, secret, redirect_url)
if token:
    spotify = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [8]:
results = spotify.current_user_top_tracks(limit = 50, time_range = 'long_term')
x = pd.DataFrame(results['items'])
working = copy.deepcopy(x)
for i in range(len(x)):
    
    artist_list = []
    if len(x.loc[i,'artists']) > 1:
        for artist in x.loc[i, 'artists']:
            artist_list.append(artist['name'])
        working.at[i,'artists'] = artist_list
    else:
        working.at[i, 'artists'] = x.loc[i, 'artists'][0]['name']

    working.at[i,'album'] = x.loc[i,'album']['name']
    
working = working.drop(['available_markets', 'disc_number', 'explicit', 'external_ids', 'external_urls', 'href','preview_url', 'track_number', 'type', 'uri','is_local','duration_ms' ], axis = 1)
working

,album,artists,id,name,popularity
0,Vagrant,"[Feint, Veela]",1M85o2ruB9uTj68dIXj6dJ,Vagrant,54
1,UKF Drum & Bass 2016,Sigma,23L2jzpnyfxbCmliMBMGQM,Stay - Maduk Remix,40
2,Getaway (Remixes),"[Tritonal, Angel Taylor, Koven]",10EpXLXKHmNSVKvX7A5hg8,Getaway - Koven Remix,54
3,Into You (3LAU Remix),"[Ariana Grande, 3LAU]",1cfVkL84XmRON3Gl9jDXKI,Into You - 3LAU Remix,53
4,Feels (Jai Wolf Remix),"[Kiiara, Jai Wolf]",57rOXabUKgkrwHPSqzo4s5,Feels - Jai Wolf Remix,54
5,Ain't My Fault (R3hab Remix),"[Zara Larsson, R3HAB]",3YPlsrk0Voe8AOa1jyN891,Ain't My Fault - R3hab Remix,60
6,Creed II: The Album,"[Mike WiLL Made-It, A$AP Rocky, A$AP Ferg, Nic...",23dwrzeJtEEGl2U4CPWSor,"Runnin (with A$AP Rocky, A$AP Ferg & Nicki Minaj)",65
7,Close,"[IZECOLD, Molly Ann]",454IPfzFpYJsCvqbS0UDbR,Close,43
8,Crazy Love (Niko The Kid Remix),"[Audien, Deb’s Daughter, Niko The Kid]",7tdBWDVM5W5Bkx0mXsnYzW,Crazy Love - Niko The Kid Remix,23
9,Friends,"[It's Different, Drama B]",2FXreiGteQpcRuMuvaSQeX,Friends,23


In [9]:
y = spotify.current_user_playlists()['items']
playlists = pd.DataFrame(y)
playlists = playlists[['name','id']]


good_playlist_id = playlists[playlists['name'] == 'Python Test 1']
good_playlist_id = good_playlist_id.iloc[0,1]


bad_playlist_id = playlists[playlists['name'] == 'Python Test 0']
bad_playlist_id = bad_playlist_id.iloc[0,1]

In [10]:
good_uncleaned = []
for i in range(10):
    try:
        good_uncleaned.append(pd.DataFrame(spotify.playlist_tracks(good_playlist_id, offset = 100 * i)['items'])['track'])
    except:
        break
good_uncleaned_tracks = pd.concat(good_uncleaned)
good_uncleaned_tracks = good_uncleaned_tracks.reset_index(drop = True)
good_track_ids = []

for i in range(len(good_uncleaned_tracks)):
    song_id = good_uncleaned_tracks[i]['id']
    good_track_ids.append(song_id)
good_feats = []
    
for i in range(10):
 
    try:
        good_feats.append(pd.DataFrame(spotify.audio_features(good_track_ids[i*100:(i+1)*100])))
    except:
        print(i)
        good_feats.append(pd.DataFrame(spotify.audio_features(good_track_ids[i*100:])))
        break

good_audio_feats = pd.concat(good_feats)
good_audio_feats = good_audio_feats.drop(0, axis = 1)
good_audio_feats = good_audio_feats.reset_index(drop = True)
good_audio_feats = good_audio_feats.dropna()
good_audio_feats['target'] = 1

In [11]:
len(good_audio_feats)

345

In [12]:
bad_uncleaned  = []

for i in range(10):
    try:
        bad_uncleaned.append(pd.DataFrame(spotify.playlist_tracks(bad_playlist_id, offset = 100*i)['items'])['track'])
    except:
        break

bad_uncleaned_tracks = pd.concat(bad_uncleaned)

bad_uncleaned_tracks = bad_uncleaned_tracks.reset_index(drop = True)
bad_track_ids = []

for i in range(len(bad_uncleaned_tracks)):
    song_id = bad_uncleaned_tracks[i]['id']
    bad_track_ids.append(song_id)
bad_feats = []
for i in range(10):
 
    try:
        bad_feats.append(pd.DataFrame(spotify.audio_features(bad_track_ids[i*100:(i+1)*100])))
    except:
        print(i)
        bad_feats.append(pd.DataFrame(spotify.audio_features(bad_track_ids[i*100:])))
        break

bad_audio_feats = pd.concat(bad_feats)
bad_audio_feats = bad_audio_feats.drop(0, axis = 1)
bad_audio_feats = bad_audio_feats.reset_index(drop = True)
bad_audio_feats = bad_audio_feats.dropna()
bad_audio_feats['target'] = 0



In [13]:
all_audio_feats = pd.concat([good_audio_feats, bad_audio_feats])
all_audio_feats = all_audio_feats.reset_index(drop = True)
from sklearn.model_selection import train_test_split
train, test = train_test_split(all_audio_feats, test_size = 0.2)

In [14]:
all_audio_feats

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,target
0,0.06770,https://api.spotify.com/v1/audio-analysis/1Tn8...,0.629,179000.0,0.915,1Tn8Se0b2NCV7QsxVW4rel,0.000000,5.0,0.3600,-1.524,1.0,0.1160,122.970,4.0,https://api.spotify.com/v1/tracks/1Tn8Se0b2NCV...,audio_features,spotify:track:1Tn8Se0b2NCV7QsxVW4rel,0.529,1
1,0.18300,https://api.spotify.com/v1/audio-analysis/1UUM...,0.844,237923.0,0.459,1UUMw2PSv5TddNzuqu4MVY,0.000005,9.0,0.0772,-9.481,0.0,0.1880,129.998,4.0,https://api.spotify.com/v1/tracks/1UUMw2PSv5Td...,audio_features,spotify:track:1UUMw2PSv5TddNzuqu4MVY,0.335,1
2,0.05180,https://api.spotify.com/v1/audio-analysis/4dtT...,0.822,169831.0,0.792,4dtTsxiztEoD9SQLBhifal,0.000001,11.0,0.1240,-4.471,0.0,0.1240,117.984,4.0,https://api.spotify.com/v1/tracks/4dtTsxiztEoD...,audio_features,spotify:track:4dtTsxiztEoD9SQLBhifal,0.842,1
3,0.00945,https://api.spotify.com/v1/audio-analysis/0WSj...,0.586,160000.0,0.896,0WSjVLITbxV6kVXndziQDm,0.836000,5.0,0.1210,-4.132,0.0,0.3720,125.744,4.0,https://api.spotify.com/v1/tracks/0WSjVLITbxV6...,audio_features,spotify:track:0WSjVLITbxV6kVXndziQDm,0.103,1
4,0.03640,https://api.spotify.com/v1/audio-analysis/6ykd...,0.617,159818.0,0.762,6ykdXpEG7cTSOfRSBv1ROA,0.000000,11.0,0.2350,-5.061,0.0,0.0414,109.959,4.0,https://api.spotify.com/v1/tracks/6ykdXpEG7cTS...,audio_features,spotify:track:6ykdXpEG7cTSOfRSBv1ROA,0.390,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047,0.16500,https://api.spotify.com/v1/audio-analysis/2TWS...,0.774,209957.0,0.912,2TWSilrWcTrgVKcP8kfEfg,0.000000,2.0,0.7450,-2.787,0.0,0.3470,150.083,4.0,https://api.spotify.com/v1/tracks/2TWSilrWcTrg...,audio_features,spotify:track:2TWSilrWcTrgVKcP8kfEfg,0.821,0
1048,0.23600,https://api.spotify.com/v1/audio-analysis/43wO...,0.556,283667.0,0.804,43wOnyLJcwYfKVNxXZWPDz,0.000000,8.0,0.3800,-3.919,1.0,0.0421,116.973,3.0,https://api.spotify.com/v1/tracks/43wOnyLJcwYf...,audio_features,spotify:track:43wOnyLJcwYfKVNxXZWPDz,0.413,0
1049,0.02220,https://api.spotify.com/v1/audio-analysis/0M45...,0.613,200253.0,0.917,0M45YYpFECO0xvXlKgQ7sE,0.000006,5.0,0.1050,-2.549,1.0,0.0540,131.007,4.0,https://api.spotify.com/v1/tracks/0M45YYpFECO0...,audio_features,spotify:track:0M45YYpFECO0xvXlKgQ7sE,0.538,0
1050,0.03280,https://api.spotify.com/v1/audio-analysis/41NS...,0.597,236787.0,0.820,41NSrKm33WLGX1xIJgU6Np,0.000000,5.0,0.1190,-2.529,1.0,0.0582,119.969,4.0,https://api.spotify.com/v1/tracks/41NSrKm33WLG...,audio_features,spotify:track:41NSrKm33WLGX1xIJgU6Np,0.576,0


In [15]:
features = ["danceability", "loudness", "valence", "energy", "instrumentalness", "acousticness", "key", "speechiness", 'liveness',"tempo"]

x_train = train[features]
y_train = train["target"]
x_test = test[features]
y_test = test["target"]

ss = StandardScaler()
x_train_scale = ss.fit_transform(x_train)
x_test_scale = ss.transform(x_test)

In [16]:
grid_params = {'n_neighbors':[ 5, 7, 9 , 15, 21, 23],
              'weights':['uniform', 'distance'],
              'metric':['euclidean', 'manhattan']}
gs = GridSearchCV(KNeighborsClassifier(), grid_params)

gs_results = gs.fit(x_train_scale, y_train)

In [17]:
gs_results.best_params_

{'metric': 'manhattan', 'n_neighbors': 21, 'weights': 'distance'}

In [18]:
knn = KNeighborsClassifier(23, weights = 'uniform', metric = 'manhattan')
knn.fit(x_train_scale, y_train)
knn_pred = knn.predict(x_test_scale)
score = accuracy_score(y_test, knn_pred) * 100
print("Accuracy using Knn Tree: ", round(score, 1), "%")

Accuracy using Knn Tree:  75.8 %


In [ ]:
max_depth = [int(x) for x in np.linspace(10, 110, num = 4)]


In [ ]:
grid_params_rf = {'max_features':["auto", "sqrt", "log2"],
              'n_estimators':[int(x) for x in np.linspace(start = 100, stop = 2000, num = 30)],
             'max_depth':max_depth,
               'min_samples_leaf' : [1, 2],
              'bootstrap': [True, False],
              'min_samples_split' : [2],}
gs_rf = GridSearchCV(RandomForestClassifier(), grid_params_rf)

gs_results_rf = gs_rf.fit(x_train, y_train)

In [ ]:
gs_results_rf.best_params_

In [19]:
clf = RandomForestClassifier(n_estimators = 427, min_samples_leaf=2, min_samples_split=2, max_depth = 10, max_features='log2', bootstrap=False)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
score = accuracy_score(y_test, y_pred) * 100
print("Accuracy using Decision Tree: ", round(score, 1), "%")

Accuracy using Decision Tree:  76.8 %


In [20]:
song_db = pd.read_csv('./data.csv')

FileNotFoundError: [Errno 2] File ./data.csv does not exist: './data.csv'

In [21]:
song_db_features = song_db[features]
a = list(knn.predict(song_db_features))
b = list(clf.predict(song_db_features))
song_db['result_rf'] = b
song_db['result_knn'] = a
x = song_db[features + ['result_rf','result_knn','id']]
good_songs = x[(x['result_rf'] == 1) & x['result_knn'] == 1]
len(good_songs)

NameError: name 'song_db' is not defined

In [ ]:
x = song_db[features + ['result_rf','result_knn','id']]
good_songs = x[(x['result_rf'] == 1) & x['result_knn'] == 1]
len(good_songs)


In [ ]:
spotify.track('764wHEOb2yPgsJHmU0J4ZS')['album']['artists'][0]['name']

In [ ]:
len(y_train)

In [ ]:
tth = playlists[playlists['name'] == 'Today\'s Top Hits']
tth_id = tth.iloc[0,1]
tth_uncleaned_tracks = pd.DataFrame(spotify.playlist_tracks(tth_id)['items'])['track']
tth_track_ids = []
for i in range(len(tth_uncleaned_tracks)):
    song_id = tth_uncleaned_tracks[i]['id']
    tth_track_ids.append(song_id)
tth_audio_feats = pd.DataFrame(spotify.audio_features(tth_track_ids))
tth_test = tth_audio_feats[features]
a = list(knn.predict(tth_test))
tth_audio_feats['result'] = a

today = tth_audio_feats[features + ['result','id']]

track_names = spotify.tracks(list(today['id']))

names_list = []
print(len(track_names))

for i in range(len(today)):
    
    names_list.append(track_names['tracks'][i]['name'])


today['song name'] = names_list

In [ ]:
spotify.tracks(list(good_songs['id'])[0:50])

In [ ]:
name = 'ML'
spotify.user_playlist_create(username, name, public=True, collaborative=False, description='')
for item in spotify.current_user_playlists()['items']:
    if item['name'] == name:
        new_playlist_id = item['id']
new_playlist_id

In [ ]:
songs_to_add = list(good_songs[:100]['id'])
spotify.playlist_add_items(new_playlist_id, songs_to_add)

In [ ]:
spotify.playlist_add_items(new_playlist_id, songs_to_add)

In [ ]:
spotify.recommendation_genre_seeds()

In [ ]:
wanted_genres = ['edm','dance']

In [ ]:
x = spotify.recommendations(seed_artists=None, seed_genres=wanted_genres, seed_tracks=None, limit=100)

In [ ]:
test_ids = []
for i in range(100):
    test_ids.append(x['tracks'][i]['id'])
  

In [ ]:
test_df = pd.DataFrame(spotify.audio_features(test_ids))

In [ ]:
song_test_features = test_df[features]
a = list(knn.predict(song_test_features))
b = list(clf.predict(song_test_features))
test_df['result_rf'] = b
test_df['result_knn'] = a
y = test_df[features + ['result_rf','result_knn','id']]
good_songs = y[(y['result_rf'] == 1) | y['result_knn'] == 1]


In [22]:
test_song_list = []
test_ids = []
for i in range(len(good_track_ids)):

    x = spotify.recommendations(seed_artists=None, seed_genres=None, seed_tracks=[good_track_ids[i]], limit=3)
    for j in range(len(x['tracks'])):
        s_id = x['tracks'][j]['id']
        test_ids.append(pd.DataFrame(spotify.audio_features(s_id)))

y = pd.concat(test_ids)
y = y.reset_index(drop = True)
z2 = y.drop_duplicates()
song_test_features = z2[features]
a = list(knn.predict(song_test_features))
b = list(clf.predict(song_test_features))
z2['result_rf'] = b
z2['result_knn'] = a
result = z2[features + ['result_rf','result_knn','id']]
good_songs = result[(result['result_rf'] == 1) & result['result_knn'] == 1]
test_song_list.append(good_songs)


<ipython-input-22-4f5d76e8c01b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z2['result_rf'] = b
<ipython-input-22-4f5d76e8c01b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z2['result_knn'] = a


In [29]:
good_songs = good_songs.sample(25)

In [27]:
name = 'ML'
spotify.user_playlist_create(username, name, public=True, collaborative=False, description='')
for item in spotify.current_user_playlists()['items']:
    if item['name'] == name:
        new_playlist_id = item['id']
new_playlist_id

'5IqV5S3o7g1io7R62gfS7o'

In [31]:
good_songs

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,liveness,tempo,result_rf,result_knn,id
341,0.645,-5.439,0.424,0.795,0.000010,0.03340,1,0.0343,0.0890,127.959,1,1,1gwwks3lIsvJq5pDJ82fKK
851,0.628,-4.875,0.350,0.681,0.000007,0.00305,9,0.0545,0.6750,104.997,1,1,0viHtyuhVNnkIACRjh08N4
578,0.601,-2.425,0.377,0.801,0.000000,0.13900,5,0.0473,0.1030,126.955,1,1,5BuiZmp7H6hoUN5Fw2lUIp
645,0.425,-5.098,0.215,0.742,0.000770,0.00772,11,0.0386,0.1120,127.978,1,1,6SKkN6yuQFnzMEHMQR3sWw
760,0.448,-3.162,0.511,0.856,0.000020,0.02110,5,0.0537,0.2000,127.977,1,1,5cImOblhS97lw6MvbPupUu
577,0.680,-4.456,0.552,0.912,0.000019,0.01440,11,0.0421,0.2140,125.938,1,1,6t2d3kJooJVtBK9GPSuwiR
336,0.661,-5.299,0.387,0.758,0.000000,0.07970,7,0.0864,0.1060,114.098,1,1,1LjxmkW3xRQP2HzswhYyyc
844,0.619,-5.584,0.138,0.560,0.000000,0.04100,8,0.0372,0.1180,119.946,1,1,1w5GiowT8eXagycUBqcz2g
918,0.579,-3.324,0.270,0.868,0.000000,0.05330,8,0.0388,0.3530,128.035,1,1,7GKyIAKAXEt5W5rAzDCW6a
373,0.519,-3.101,0.416,0.931,0.000020,0.00770,3,0.0448,0.0784,127.913,1,1,2RYICR8IFZNaD2UA1W3ONT


In [30]:
songs_by_100 = int(np.ceil(len(good_songs)/100))
songs_by_100

1

In [34]:
for i in range(songs_by_100):
    print(i)
    if i < (songs_by_100 - 1):
        songs_to_add = list(good_songs['id'][i*100:100*(i+1)])
        spotify.playlist_add_items(new_playlist_id, songs_to_add)
    else:
        songs_to_add = list(good_songs['id'][i*100:])
        spotify.playlist_add_items(new_playlist_id, songs_to_add)

0


In [33]:
len(list(good_songs['id'][0:]))

25

In [ ]:
z2 = y.drop_duplicates()
len(z2)